# Multi-objective Robust Optimization
A robust optimization method for finding policies under deep uncertainty across scenarios.


### sources:
https://doi.org/10.1007/s10584-014-1210-4
https://emaworkbench.readthedocs.io/en/latest/examples/robust_optimization_lake_model_dps.html
https://github.com/WKSu/Model-based-decision-making/blob/ba6a6f2b9f7f32b56d7e11c9caeaedf5562deea4/MORO.py

### Problem formulation

    0. Total cost, and casualties
    1. Expected damages, costs, and casualties
    2. expected damages, dike investment costs, rfr costs, evacuation cost, and casualties
    3. costs and casualties disaggregated over dike rings, and room for the river and evacuation costs
    4. Expected damages, dike investment cost and casualties disaggregated over dike rings and room for the river and evacuation costs
    5. disaggregate over time and space

First, packages are imported.

In [8]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pickle

from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
    util,
    ScalarOutcome,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time
from ema_workbench.em_framework.samplers import sample_uncertainties

ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

In [18]:
dike_model, planning_steps = get_model_for_problem_formulation(5)
dike_model.outcomes.keys()

odict_keys(['A.1_Expected Annual Damage', 'A.1_Dike Investment Costs', 'A.1_Expected Number of Deaths', 'A.2_Expected Annual Damage', 'A.2_Dike Investment Costs', 'A.2_Expected Number of Deaths', 'A.3_Expected Annual Damage', 'A.3_Dike Investment Costs', 'A.3_Expected Number of Deaths', 'A.4_Expected Annual Damage', 'A.4_Dike Investment Costs', 'A.4_Expected Number of Deaths', 'A.5_Expected Annual Damage', 'A.5_Dike Investment Costs', 'A.5_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs'])

In [11]:
def robust_analysis():

    # choose problem formulation number, between 0-5
    # each problem formulation has its own list of outcomes
    dike_model, planning_steps = get_model_for_problem_formulation(0)

    dike_model.outcomes


    def signal_to_noise(data):
        mean = np.mean(data)
        std = np.std(data)
        sn = mean / std
        return sn

    MAXIMIZE = ScalarOutcome.MAXIMIZE
    MINIMIZE = ScalarOutcome.MINIMIZE
    robustness_functions = [
        ScalarOutcome("mean p", kind=MINIMIZE, variable_name="4_RfR 2", function=np.mean),
        ScalarOutcome("std p", kind=MINIMIZE, variable_name="4_RfR 2", function=np.std),
        ScalarOutcome("sn reliability", kind=MAXIMIZE, variable_name="4_RfR 2", function=signal_to_noise),
    ]

    n_scenarios = 10
    scenarios = sample_uncertainties(dike_model, n_scenarios)
    nfe = 1000

    with MultiprocessingEvaluator(dike_model) as evaluator:
        evaluator.robust_optimize(
            robustness_functions,
            scenarios,
            nfe=nfe,
            epsilons=[0.1] * len(robustness_functions),
            population_size=5,
        )

In [12]:
robust_analysis()

  0%|                                                 | 0/1000 [00:00<?, ?it/s]

KeyError: '4_RfR 2'

In [19]:
test = pd.read_pickle('test.pkl')
test

,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,...,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,mean p,std p,sn reliability
0,0,0,1,1,1,0,0,1,1,0,...,2,10,0,9,2,9,3,0.004025,0.009496,0.0
